# 🧪 ROS 2 Bag File Diagnostic Suite for SLAM & Navigation2

In [ ]:
bag_path = "/absolute/path/to/your_bag_folder"  # Change this to your actual bag path

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from rosbag2_py import SequentialReader, StorageOptions, ConverterOptions
from rclpy.serialization import deserialize_message
from nav_msgs.msg import Odometry
from sensor_msgs.msg import Imu, LaserScan
import numpy as np


In [ ]:

def list_all_topics(bag_path):
    storage_options = StorageOptions(uri=bag_path, storage_id='sqlite3')
    converter_options = ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr')
    reader = SequentialReader()
    reader.open(storage_options, converter_options)
    return reader.get_all_topics_and_types()

list_all_topics(bag_path)


In [ ]:

def read_odom_from_bag(bag_path):
    storage_options = StorageOptions(uri=bag_path, storage_id='sqlite3')
    converter_options = ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr')
    reader = SequentialReader()
    reader.open(storage_options, converter_options)

    odom_data = []
    while reader.has_next():
        topic, data, timestamp = reader.read_next()
        if topic == "/odom":
            msg = deserialize_message(data, Odometry)
            odom_data.append({
                't': timestamp * 1e-9,
                'x': msg.pose.pose.position.x,
                'y': msg.pose.pose.position.y,
                'v_x': msg.twist.twist.linear.x,
                'angular_z': msg.twist.twist.angular.z
            })
    return pd.DataFrame(odom_data)


In [ ]:
df_odom = read_odom_from_bag(bag_path)
df_odom.head()

In [ ]:

plt.figure(figsize=(6,6))
plt.plot(df_odom['x'], df_odom['y'], label='Trajectory')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.title('Robot Path from /odom')
plt.grid()
plt.axis("equal")
plt.legend()
plt.show()


In [ ]:

plt.figure(figsize=(10, 4))
plt.plot(df_odom['t'], df_odom['v_x'], label='Linear X')
plt.plot(df_odom['t'], df_odom['angular_z'], label='Angular Z')
plt.title("Odometry Velocities")
plt.xlabel("Time [s]")
plt.ylabel("Velocity (m/s or rad/s)")
plt.legend()
plt.grid()
plt.show()


In [ ]:

def read_imu_from_bag(bag_path):
    storage_options = StorageOptions(uri=bag_path, storage_id='sqlite3')
    converter_options = ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr')
    reader = SequentialReader()
    reader.open(storage_options, converter_options)

    imu_data = []
    while reader.has_next():
        topic, data, timestamp = reader.read_next()
        if topic == "/imu/data":
            msg = deserialize_message(data, Imu)
            imu_data.append({
                't': timestamp * 1e-9,
                'angular_z': msg.angular_velocity.z,
                'acc_x': msg.linear_acceleration.x,
                'acc_y': msg.linear_acceleration.y
            })
    return pd.DataFrame(imu_data)


In [ ]:
df_imu = read_imu_from_bag(bag_path)
df_imu.head()

In [ ]:

plt.figure(figsize=(10, 4))
plt.plot(df_imu['t'], df_imu['angular_z'], label='Gyro Z')
plt.xlabel("Time [s]")
plt.ylabel("Angular Velocity Z (rad/s)")
plt.title("IMU Angular Velocity Over Time")
plt.grid()
plt.legend()
plt.show()


In [ ]:

def read_scan_from_bag(bag_path):
    storage_options = StorageOptions(uri=bag_path, storage_id='sqlite3')
    converter_options = ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr')
    reader = SequentialReader()
    reader.open(storage_options, converter_options)

    scan_ranges = []
    while reader.has_next():
        topic, data, timestamp = reader.read_next()
        if topic == "/scan":
            msg = deserialize_message(data, LaserScan)
            mean_range = np.mean(msg.ranges)
            scan_ranges.append({
                't': timestamp * 1e-9,
                'mean_range': mean_range
            })
    return pd.DataFrame(scan_ranges)


In [ ]:
df_scan = read_scan_from_bag(bag_path)
df_scan.head()

In [ ]:

plt.figure(figsize=(10, 4))
plt.plot(df_scan['t'], df_scan['mean_range'], label='Mean Range')
plt.xlabel("Time [s]")
plt.ylabel("Mean Lidar Distance [m]")
plt.title("Mean Lidar Distance vs Time")
plt.grid()
plt.legend()
plt.show()


In [ ]:

plt.figure(figsize=(10, 4))
plt.plot(df_odom['t'], label='odom')
plt.plot(df_imu['t'], label='imu')
plt.plot(df_scan['t'], label='scan')
plt.title("Timestamps of Topics")
plt.xlabel("Index")
plt.ylabel("Time [s]")
plt.legend()
plt.grid()
plt.show()
